# Exploration of the flight data

This notebook can be used to visualize the data once simulated.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from datetime import datetime, timedelta

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.lines import Line2D
from matplotlib.gridspec import GridSpec
from tqdm import tqdm

from dcm import dcm
from variable_correspondence import VARIABLE_DESCRIPTION

## Data validity check

### NaN values

In [4]:
folder_path = (
    Path.home()
    / "Documents"
    / "data"
    / "Safire_meghatropique"
    / "simulations"
    / "oop_control"
    / "light"
)
file_paths = sorted(folder_path.glob("*.csv"))
for fp in file_paths:
   data = pd.read_csv(fp).iloc[:-1, :]
   print(f"Nan data for {fp.name}: {data.isna().sum().sum()}")

## Data loading

In [5]:
file_names = list(folder_path.glob("*.csv"))
file_id=21
#file_name = "F20_1Hz-MEGHA-2011_base_v3_20111108_fs110041_simulated.csv"
file_name = file_names[file_id].name
file_path = folder_path / file_name

data = pd.read_csv(file_path).iloc[:-1:20,:] # resampling at 1Hz
data = data.rename(columns=VARIABLE_DESCRIPTION)

In [ ]:
all_data = pd.DataFrame()
for file_path in tqdm(file_paths):
    df = pd.read_csv(file_path).iloc[:-1:10,:] # resampling at 2Hz to reduce memory usage
    df = df.rename(columns=VARIABLE_DESCRIPTION)
    df["file_name"] = file_path.name
    all_data = pd.concat([all_data, df], axis=0)

### Highest Angle of attack

In order to check the hypothesis of flights without stall

In [ ]:
print(f"Maximal angle of attack : {all_data['Angle of attack, alphar, rad'].max() * 180/np.pi:.2f}°")

### Total flight duration

In [ ]:
print(f"Total simulation time: {all_data.shape[0]/2/3600:.2f} hours")

## Flight data display

### Commands

In [ ]:
fig = sp.make_subplots(
    rows=2,
    cols=2,
    subplot_titles=(
        "Pitch Test Inputs",
        "Lateral-Directional Test Inputs",
        "Throttle Test Inputs",
        "Flap Test Inputs",
    ),
    shared_xaxes=True,
)

# Subplot 1
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Elevator, dEr, rad, positive: trailing edge down"],
        name="Elevator, dE",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Stabilator, dSr, rad"],
        name="Stabilator, dS",
    ),
    row=1,
    col=1,
)
fig.update_xaxes(title_text="Time, s", row=1, col=1)
fig.update_yaxes(title_text="Elevator (blue), Stabilator (green), deg", row=1, col=1)

# Subplot 2
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Aileron, dAr, rad, positive: left trailing edge down"],
        name="Aileron, dA",
    ),
    row=1,
    col=2,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Rudder, dRr, rad, positive: trailing edge left"],
        name="Rudder, dR",
    ),
    row=1,
    col=2,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Asymmetric Spoiler, dASr, rad"],
        name="Asymmetric Spoiler, dAS",
    ),
    row=1,
    col=2,
)
fig.update_xaxes(title_text="Time, s", row=1, col=2)
fig.update_yaxes(
    title_text="Aileron (blue), Rudder (green), Asymmetric Spoiler (red), deg",
    row=1,
    col=2,
)

# Subplot 3
fig.add_trace(
    go.Scatter(x=data["Time, s"], y=data["Throttle, dT, %"], name="Throttle Setting"),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(x=data["Time, s"], y=data["Turbulent conditions, Turbulence, boolean"], name="Turbulence"),
)
fig.update_xaxes(title_text="Time, s", row=2, col=1)
fig.update_yaxes(title_text="Throttle Setting", row=2, col=1)

# Subplot 4
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Position of the flaps, fl, rad"],
        name="Flap, deg",
    ),
    row=2,
    col=2,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Flap command, dFr, rad"],
        name="Flap command, deg",
    ),
    row=2,
    col=2,
)
fig.update_xaxes(title_text="Time, s", row=2, col=2)
fig.update_yaxes(title_text="Flap, deg", row=2, col=2)

fig.update_layout(showlegend=True)

fig.show()

### Altitude control

In [ ]:
fig = sp.make_subplots(
    rows=2,
    cols=1,
    subplot_titles=("Altitude", "Y-axis angles", "Commands"),
    shared_xaxes=True,
    specs=[[{"secondary_y": True}], [{"secondary_y": True}]],
)

fig.add_trace(
    go.Scatter(
        x=data["Time, s"], y=data["Target altitude, th, m"], name="Target altitude, m"
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=-data["Negative altitude WRT Earth, ze=-h, m"],
        name="Altitude, m",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Target flight path angle gamma, tgammar, rad"],
        name="Target flight path angle gamma, deg",
    ),
    row=1,
    col=1,
    secondary_y=True,
)
fig.update_yaxes(title_text="Altitude, m", row=1, col=1)
fig.update_yaxes(
    title_text="Target flight path angle gamma, deg", row=1, col=1, secondary_y=True
)


fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Pitch angle of body WRT Earth, thetar, rad"],
        name="Pitch angle, theta, deg",
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Target flight path angle gamma, tgammar, rad"],
        name="Target flight path angle gamma, deg",
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578
        * data["Flight path angle, gammar, rad"],
        name="Flight path angle gamma, deg",
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Angle of attack, alphar, rad"],
        name="Angle of attack alpha, deg",
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Elevator, dEr, rad, positive: trailing edge down"],
        name="Elevator, dEr, deg, positive: trailing edge down",
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Stabilator, dSr, rad"],
        name="Stabilator, dSr, deg",
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=data["Z load factor, nz"],
        name="Z load factor"
    ),
    row=2,
    col=1,
    secondary_y=True
)

### Speed control

In [ ]:
fig = sp.make_subplots(
    rows=3,
    cols=1,
    subplot_titles=("Speed", "flight path angle gamma"),
    shared_xaxes=True,
    specs=[
        [{"secondary_y": True}], 
        [{"secondary_y": False}], 
        [{"secondary_y": False}]
    ],
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=data["Target air speed, ttas, m/s"],
        name="Target air speed, m/s",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=data["True air speed, tas, m/s"],
        name="True air speed, m/s",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=data["Throttle, dT, %"],
        name="Throttle, dT, %",
    ),
    row=1,
    col=1,
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Flight path angle, gammar, rad"],
        name="Flight path angle gamma, deg",
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=data["Position of the landing gear, ge, rad"],
        name="Position of the gear, %",
    ),
    row=3,
    col=1,
)

### Lateral control
The yaw and roll are présented on the same graphs in order to see if dutch roll appears

In [ ]:
fig  = sp.make_subplots(
    rows=3,
    cols=1,
    subplot_titles=("Roll", "Yaw"),
    shared_xaxes=True,
    specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": False}]],
)

fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Target route Xi, txir, rad"],
        name="Target route Xi, rad",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Target roll angle phi, tphir, rad"],
        name="Target roll angle phi, deg",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Roll angle of body WRT Earth, phir, rad"],
        name="Roll angle phi, deg",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Aileron, dAr, rad, positive: left trailing edge down"],
        name="Aileron, dAr, deg, positive: left trailing edge down",
    ),
    row=1,
    col=1,
)


fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Target route Xi, txir, rad"],
        name="Target route Xi, deg",
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Route angle, xir, rad"],
        name="Route angle, xi, deg",
    ),
    row=2,
    col=1,
)


fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Sideslip angle, betar, rad"],
        name="Sideslip angle beta, deg",
    ),
    row=3,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Rudder, dRr, rad, positive: trailing edge left"],
        name="Rudder, dRr, deg, positive: trailing edge left",
    ),
    row=3,
    col=1,
)

### Effects of icing

In [ ]:
fig = sp.make_subplots(
    rows=3,
    cols=1,
    subplot_titles=("Givrage", "Macro effects", "Coefficients aérodynamiques"),
    shared_xaxes=True,
    specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}]],
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=data["Icing conditions, Icing, boolean"],
        name="Icing",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=data["Proportion of icing on the aircraft, ice, %"],
        name="Proportion of icing on the aircraft, ice, %",
    ),
    row=1,
    col=1
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=57.29578 * data["Angle of attack, alphar, rad"],
        name="Angle of attack, alpha, deg",
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=data["Throttle, dT, %"],
        name="Throttle, dT, %",
    ),
    row=2,
    col=1,
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=data["Lift coefficient, cz"],
        name="Lift coefficient",
    ),
    row=3,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=data["Time, s"],
        y=data["Drag coefficient, cx"],
        name="Drag coefficient",
    ),
    row=3,
    col=1,
)

### Mass variation during the flight

In [ ]:
data[["Mass of the aircraft, m, kg"]].plot()

### Speed

In [ ]:
# Création des sous-fenêtres
fig = sp.make_subplots(rows=2, cols=2, subplot_titles=("Forward Body-Axis Component of Inertial Velocity, u",
                                                      "Side Body-Axis Component of Inertial Velocity, v",
                                                      "Normal Body-Axis Component of Inertial Velocity, w",
                                                      "Body-Axis Component of Inertial Velocity"))

# Subplot 1
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Body-axis x inertial velocity, ub, m/s'], name='Axial Velocity (u)'),
              row=1, col=1)
fig.update_xaxes(title_text='Time, s', row=1, col=1)
fig.update_yaxes(title_text='Axial Velocity (u), m/s', row=1, col=1)

# Subplot 2
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Body-axis y inertial velocity, uv, m/s'], name='Side Velocity (v)'),
              row=1, col=2)
fig.update_xaxes(title_text='Time, s', row=1, col=2)
fig.update_yaxes(title_text='Side Velocity (v), m/s', row=1, col=2)

# Subplot 3
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Body-axis z inertial velocity, wb, m/s'], name='Normal Velocity (w)'),
              row=2, col=1)
fig.update_xaxes(title_text='Time, s', row=2, col=1)
fig.update_yaxes(title_text='Normal Velocity (w), m/s', row=2, col=1)

# Subplot 4
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Body-axis x inertial velocity, ub, m/s'], name='Axial velocity, u'),
              row=2, col=2)
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Body-axis y inertial velocity, vb, m/s'], name='Side velocity, v'),
              row=2, col=2)
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Body-axis z inertial velocity, wb, m/s'], name='Normal velocity, w'),
              row=2, col=2)
fig.update_xaxes(title_text='Time, s', row=2, col=2)
fig.update_yaxes(title_text='u (blue), v (green), w (red), m/s', row=2, col=2)

# Mise en page des sous-fenêtres
fig.update_layout(showlegend=True)

# Affichage des sous-fenêtres
fig.show()


### Trajectories

In [ ]:
# Création des sous-fenêtres
fig = sp.make_subplots(rows=3, cols=2, subplot_titles=("North Location, x",
                                                      "East Location, y",
                                                      "Altitude, -z",
                                                      "Altitude vs. Ground Range",
                                                      "Ground Track, North vs. East",
                                                      "3D Flight Path"), specs=[[{}, {}],[{},{}],[{},{"type": "scene"}]])

# Subplot 1
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['North position of center of mass WRT Earth, xe, m'], name='North (x)'),
              row=1, col=1)
fig.update_xaxes(title_text='Time, s', row=1, col=1)
fig.update_yaxes(title_text='North (x), m', row=1, col=1)

# Subplot 2
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['East position of center of mass WRT Earth, ye, m'], name='East (y)'),
              row=1, col=2)
fig.update_xaxes(title_text='Time, s', row=1, col=2)
fig.update_yaxes(title_text='East (y), m', row=1, col=2)

# Subplot 3
fig.add_trace(go.Scatter(x=data['Time, s'], y=-data['Negative altitude WRT Earth, ze=-h, m'], name='Altitude (-z)'),
              row=2, col=1)
fig.add_trace(go.Scatter(x=data['Time, s'][:-1], y=data['Target altitude, th, m'], name='target altitude'),
              row=2, col=1)
fig.update_xaxes(title_text='Time, s', row=2, col=1)
fig.update_yaxes(title_text='Altitude (-z), m', row=2, col=1)
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))

# Subplot 4
fig.add_trace(go.Scatter(x=(data['North position of center of mass WRT Earth, xe, m']**2 + data['East position of center of mass WRT Earth, ye, m']**2)**0.5,
                         y=-data['Negative altitude WRT Earth, ze=-h, m'], name='Altitude'),
              row=2, col=2)
fig.update_xaxes(title_text='Ground Range, m', row=2, col=2)
fig.update_yaxes(title_text='Altitude, m', row=2, col=2)

# Subplot 5
fig.add_trace(go.Scatter(x=data['North position of center of mass WRT Earth, xe, m'], y=data['East position of center of mass WRT Earth, ye, m'], name='Ground Track'),
              row=3, col=1)
fig.update_xaxes(title_text='North, m', row=3, col=1)
fig.update_yaxes(title_text='East, m', row=3, col=1)

# Subplot 6
fig.add_trace(go.Scatter3d(x=data['North position of center of mass WRT Earth, xe, m'],
                           y=data['East position of center of mass WRT Earth, ye, m'],
                            z=-data['Negative altitude WRT Earth, ze=-h, m'],
                            name='3D Flight Path'),
                row=3, col=2)

### Rotation rates

In [ ]:
# Création des sous-fenêtres
fig = sp.make_subplots(rows=2, cols=2, subplot_titles=("Body-Axis Roll Component of Inertial Rate, p",
                                                      "Body-Axis Pitch Component of Inertial Rate, q",
                                                      "Body-Axis Yaw Component of Inertial Rate, r",
                                                      "Body-Axis Inertial Rate Vector Components"))

# Subplot 1
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Body axis roll rate, pr, rad/s']*57.29578, name='Roll Rate (p)'),
              row=1, col=1)
fig.update_xaxes(title_text='Time, s', row=1, col=1)
fig.update_yaxes(title_text='Roll Rate (p), deg/s', row=1, col=1)

# Subplot 2
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Body axis pitch rate, qr, rad/s']*57.29578, name='Pitch Rate (q)'),
              row=1, col=2)
fig.update_xaxes(title_text='Time, s', row=1, col=2)
fig.update_yaxes(title_text='Pitch Rate (q), deg/s', row=1, col=2)

# Subplot 3
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Body axis yaw rate, rr, rad/s']*57.29578, name='Yaw Rate (r)'),
              row=2, col=1)
fig.update_xaxes(title_text='Time, s', row=2, col=1)
fig.update_yaxes(title_text='Yaw Rate (r), deg/s', row=2, col=1)

# Subplot 4
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Body axis roll rate, pr, rad/s']*57.29578, name='Roll rate, p'),
              row=2, col=2)
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Body axis pitch rate, qr, rad/s']*57.29578, name='Pitch rate, q'),
              row=2, col=2)
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Body axis yaw rate, rr, rad/s']*57.29578, name='Yaw rate, r'),
              row=2, col=2)
fig.update_xaxes(title_text='Time, s', row=2, col=2)
fig.update_yaxes(title_text='p (blue), q (green), r (red), deg/s', row=2, col=2)
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))

# Affichage de la figure
fig.show()


### Angles

In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go
import pandas as pd

# Supposons que vous ayez déjà importé les données dans un DataFrame pandas appelé "data"

# Création des sous-fenêtres
fig = sp.make_subplots(rows=2, cols=2, subplot_titles=("Earth-Relative Roll Attitude",
                                                      "Earth-Relative Pitch Attitude",
                                                      "Earth-Relative Yaw Attitude",
                                                      "Euler Angles"))

# Subplot 1
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Roll angle of body WRT Earth, phir, rad']*57.29578, name='Roll Angle (phi)'),
              row=1, col=1)
fig.update_xaxes(title_text='Time, s', row=1, col=1)
fig.update_yaxes(title_text='Roll Angle (phi), deg', row=1, col=1)

# Subplot 2
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Pitch angle of body WRT Earth, thetar, rad']*57.29578, name='Pitch Angle (theta)'),
              row=1, col=2)
fig.update_xaxes(title_text='Time, s', row=1, col=2)
fig.update_yaxes(title_text='Pitch Angle (theta), deg', row=1, col=2)

# Subplot 3
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Yaw angle of body WRT Earth, psir, rad']*57.29578, name='Yaw Angle (psi)'),
              row=2, col=1)
fig.update_xaxes(title_text='Time, s', row=2, col=1)
fig.update_yaxes(title_text='Yaw Angle (psi), deg', row=2, col=1)

# Subplot 4
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Roll angle of body WRT Earth, phir, rad']*57.29578, name='Roll angle, phi'),
              row=2, col=2)
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Target roll angle phi, tphir, rad']*57.29578, name='Target roll angle'),
              row=2, col=2)
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Pitch angle of body WRT Earth, thetar, rad']*57.29578, name='Pitch angle, theta'),
              row=2, col=2)
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Yaw angle of body WRT Earth, psir, rad']*57.29578, name='Yaw angle, psi'),
              row=2, col=2)
fig.update_xaxes(title_text='Time, s', row=2, col=2)
fig.update_yaxes(title_text='phi (blue), theta (green), psi (red), deg', row=2, col=2)
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))

# Affichage de la figure
fig.show()


### Airspeed

In [ ]:
# Création des sous-fenêtres
fig = sp.make_subplots(rows=3, cols=1, subplot_titles=("True AirSpeed, Vair",
                                                      "Mach Number, M",
                                                      "Dynamic Pressure, qbar"))

# Subplot 1
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['True air speed, tas, m/s'], name='True airspeed'),
              row=1, col=1)
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Target air speed, ttas, m/s'], name='Target air speed'),
              row=1, col=1)
fig.update_xaxes(title_text='Time, s', row=1, col=1)
fig.update_yaxes(title_text='Air-relative Speed, m/s', row=1, col=1)

# Subplot 2
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Mach number, mach'], name='Mach number'),
              row=2, col=1)
fig.update_xaxes(title_text='Time, s', row=2, col=1)
fig.update_yaxes(title_text='M', row=2, col=1)

# Subplot 3
dynamic_pressure = 0.5 * data['Air density, rho, kg/m^3'] * data['True air speed, tas, m/s']**2
fig.add_trace(go.Scatter(x=data['Time, s'], y=dynamic_pressure, name='Dynamic Pressure'),
              row=3, col=1)
fig.update_xaxes(title_text='Time, s', row=3, col=1)
fig.update_yaxes(title_text='qbar, N/m^2', row=3, col=1)

# Affichage de la figure
fig.show()


### Aerodynamical angles

In [ ]:
# Création des sous-fenêtres
fig = sp.make_subplots(rows=2, cols=1, subplot_titles=("Aerodynamic Angles", "Flight Path Angles"))

# Subplot 1
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Angle of attack, alphar, rad']*180/np.pi, name='Angle of Attack, alpha'),
              row=1, col=1)
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Sideslip angle, betar, rad']*180/np.pi, name='Sideslip Angle, beta'),
              row=1, col=1)
fig.update_xaxes(title_text='Time, s', row=1, col=1)
fig.update_yaxes(title_text='Angle of Attack, deg (blue), Sideslip Angle, deg (green)', row=1, col=1)

# Subplot 2
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Flight path angle, gammar, rad']*180/np.pi, name='Flight Path Angle, gamma'),
              row=2, col=1)
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Target flight path angle gamma, tgammar, rad']*180/np.pi, name='Target gamma'),
              row=2, col=1)
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Yaw angle of body WRT Earth, psir, rad']*180/np.pi, name='Heading Angle, psi'),
              row=2, col=1)
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Target route Xi, txir, rad']*180/np.pi, name='Target route angle'),
              row=2, col=1)
fig.update_xaxes(title_text='Time, s', row=2, col=1)
fig.update_yaxes(title_text='deg', row=2, col=1)

# Affichage de la figure
fig.show()


### Acceleration and load factor

In [ ]:
fig = sp.make_subplots(rows=3, cols=1, subplot_titles=("Body-Axis X Acceleration, ax",
                                                      "Body-Axis Y Acceleration, ay",
                                                      "Body-Axis Z Acceleration, az"))
fig.add_trace(go.Scatter(x=data['Time, s'], y=data['X load factor, nx'], name='X load factor'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Y load factor, ny'], name='Y load factor'),
              
                row=2, col=1)

fig.add_trace(go.Scatter(x=data['Time, s'], y=data['Z load factor, nz'], name='Z load factor'),
                row=3, col=1)

fig.update_xaxes(title_text='Time, s', row=1, col=1)
fig.update_yaxes(title_text='X load factor', row=1, col=1)
fig.update_xaxes(title_text='Time, s', row=2, col=1)
fig.update_yaxes(title_text='Y load factor', row=2, col=1)
fig.update_xaxes(title_text='Time, s', row=3, col=1)
fig.update_yaxes(title_text='Z load factor', row=3, col=1)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=57.29578*data['Angle of attack, alphar, rad'], y=data['Lift coefficient, cz'], name='Lift coefficient'))
fig.add_trace(go.Scatter(x=57.29578*data['Angle of attack, alphar, rad'], y=data['Drag coefficient, cx'], name='Drag coefficient'))
fig.show()

## Technical validation figures

This section was used to generate the figures used in the data paper.

### Lift and drag coefficients

In [ ]:
# Proportion of icing on the aircraft, ice, %
# Icing conditions, Icing, boolean
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
no_flaps_no_lgear_data = all_data[(all_data['Position of the flaps, fl, rad'] < 1e-3) & (all_data['Position of the landing gear, ge, rad'] < 1e-3) & (all_data['Lift coefficient, cz']<0.6)].iloc[::5,:]

data_icing = no_flaps_no_lgear_data[no_flaps_no_lgear_data['Proportion of icing on the aircraft, ice, %'] >0.99]
data_no_icing = no_flaps_no_lgear_data[no_flaps_no_lgear_data['Proportion of icing on the aircraft, ice, %'] <0.01]
ax.plot(data_icing['Lift coefficient, cz'], data_icing['Drag coefficient, cx'], 'o', markersize=1, alpha=0.2)
ax.plot(data_no_icing['Lift coefficient, cz'], data_no_icing['Drag coefficient, cx'], 'o', markersize=1, alpha=0.2)
ax.set_xlabel('Lift coefficient, cz')
ax.set_ylabel('Drag coefficient, cx')
legend_handles = [
    Line2D([0], [0], marker='o', color='w', label='Maximal icing', markerfacecolor='#1f77b4', markersize=5),
    Line2D([0], [0], marker='o', color='w', label='No icing', markerfacecolor='#ff7f0e', markersize=5)
]
ax.legend(handles=legend_handles)
plt.show()

fig.savefig(Path.cwd() / "figures" / "Cd_vs_Cl.pdf")

In [ ]:
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.plot(57.29578*all_data['Angle of attack, alphar, rad'], all_data['Lift coefficient, cz'], 'o', markersize=1, alpha=0.1, label='Lift coefficient')
ax.plot(57.29578*all_data['Angle of attack, alphar, rad'], all_data['Drag coefficient, cx'], 'o', markersize=1, alpha=0.1, label='Drag coefficient')
ax.set_xlabel('Angle of attack, alpha, deg')
ax.set_ylabel('Coefficients')
ax.legend()
plt.show()

fig.savefig(Path.cwd() / "figures" / "Cd_Cl_vs_alpha.pdf")

### Control of the airplane

#### Altitude

In [ ]:
file_id=1
file_name = file_names[file_id].name
file_path = folder_path / file_name

data = pd.read_csv(file_path).iloc[:-1:20,:] # resampling at 1Hz
data = data.rename(columns=VARIABLE_DESCRIPTION)
start = datetime(2024, 1, 1)
time = [start + timedelta(seconds=s) for s in data['Time, s']]

fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.plot(time, data['Target altitude, th, m'],'-.', label='Target altitude')
ax.plot(time, -data['Negative altitude WRT Earth, ze=-h, m'], label='Altitude', alpha=0.7)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.set_xlabel('Time since takeoff, HH:MM')
ax.set_ylabel('Altitude, m')
ax.legend()
fig.autofmt_xdate()
plt.show()

fig.savefig(Path.cwd() / "figures" / f"Altitude_vs_time_flight_{file_id}.pdf")

#### Airspeed

In [ ]:
file_id=7
file_name = file_names[file_id].name
file_path = folder_path / file_name

data = pd.read_csv(file_path).iloc[:-1:20,:] # resampling at 1Hz
data = data.rename(columns=VARIABLE_DESCRIPTION)

start = datetime(2024, 1, 1)
time = [start + timedelta(seconds=s) for s in data['Time, s']]

fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.plot(time, data['Target air speed, ttas, m/s'],'-.', label='Target air speed')
ax.plot(time, data['True air speed, tas, m/s'], label='True airspeed', alpha=0.7)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.set_xlabel('Time since takeoff, HH:MM')
ax.set_ylabel('Air speed, m/s')
ax.legend()
fig.autofmt_xdate()
plt.show()

fig.savefig(Path.cwd() / "figures" / f"Air_speed_vs_time_flight_{file_id}.pdf")

#### Route angle

In [ ]:
file_id = 12
file_name = file_names[file_id].name
file_path = folder_path / file_name

data = pd.read_csv(file_path).iloc[:-1:20, :]  # resampling at 1Hz
data = data.rename(columns=VARIABLE_DESCRIPTION)

# détection des sauts de valeurs
jumps_txi = [0] + [
    i
    for i in range(data.shape[0])
    if data["Target route Xi, txir, rad"].diff().abs().iloc[i] > 3.14
]
jumps_xi = [0] + [
    i
    for i in range(data.shape[0])
    if data["Route angle, xir, rad"].diff().abs().iloc[i] > 3.14
]

fig = plt.figure(figsize=(6, 4))
ax = plt.subplot(111)

for i in range(len(jumps_txi) - 1):
    time = [
        datetime(2024, 1, 1) + timedelta(seconds=s)
        for s in data["Time, s"].iloc[jumps_txi[i] : jumps_txi[i + 1]]
    ]
    ax.plot(
        time,
        data["Target route Xi, txir, rad"].iloc[jumps_txi[i] : jumps_txi[i + 1]]
        * 57.29578,
        "-.",
        color="#1f77b4",
        alpha=0.7,
    )
for i in range(len(jumps_xi) - 1):
    time = [
        datetime(2024, 1, 1) + timedelta(seconds=s)
        for s in data["Time, s"].iloc[jumps_xi[i] : jumps_xi[i + 1]]
    ]
    ax.plot(
        time,
        data["Route angle, xir, rad"].iloc[jumps_xi[i] : jumps_xi[i + 1]] * 57.29578,
        color="#ff7f0e",
        alpha=0.6,
    )

ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
ax.set_xlabel("Time since takeoff, HH:MM")
ax.set_ylabel("Route angle, deg")

legend_handles = [
    Line2D([0], [0], linestyle="-.", label="Target route angle", color="#1f77b4"),
    Line2D([0], [0], label="Route angle", color="#ff7f0e"),
]
ax.legend(handles=legend_handles)

fig.autofmt_xdate()

plt.show()

fig.savefig(Path.cwd() / "figures" / f"Route_angle_vs_time_flight_{file_id}.pdf")

### Display of a trajectory

In [ ]:
file_id=2
#file_id=3
file_name = file_names[file_id].name
file_path = folder_path / file_name

data = pd.read_csv(file_path).iloc[:-1:20,:] # resampling at 1Hz
data = data.rename(columns=VARIABLE_DESCRIPTION)

time = [datetime(2024,1,1) + timedelta(seconds=s) for s in data['Time, s']]

# font = {'family' : 'normal',
#         'weight' : 'bold',
#         'size'   : 16}

# matplotlib.rc('font', **font)

fig = plt.figure(figsize=(10,6)) # 10, 6 for paper, 3,2 for graphical abstract
gs = GridSpec(2, 2, width_ratios=[2, 1], height_ratios=[1, 1])

ax1 = fig.add_subplot(gs[:, 0])
ax1.set_title('Trajectory')
ax1.plot(data['East position of center of mass WRT Earth, ye, m']/1000, data['North position of center of mass WRT Earth, xe, m']/1000)
ax1.set_xlabel('East, km')
ax1.set_ylabel('North, km')

ax2 = fig.add_subplot(gs[0, 1])
ax2.set_title('Altitude')
ax2.plot(time, -data['Negative altitude WRT Earth, ze=-h, m'])
ax2.set_xlabel('Time since takeoff, HH:MM')
ax2.set_ylabel('Altitude, m')
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

ax3 = fig.add_subplot(gs[1, 1])
ax3.set_title('Air speed')
ax3.plot(time, data['True air speed, tas, m/s'])
ax3.set_xlabel('Time since takeoff, HH:MM')
ax3.set_ylabel('Air speed, m/s')
ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

# for ax in [ax1, ax2, ax3]:
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)

plt.tight_layout()

fig.autofmt_xdate()

plt.show()

#fig.savefig(Path.cwd() / "figures" / f"Trajectory_vs_time_flight_mini_{file_id}.jpg")
fig.savefig(Path.cwd() / "figures" / f"Trajectory_vs_time_flight_{file_id}.pdf")

### Variables value ranges

In [ ]:
var_list = [
    "True air speed, tas, m/s",
    "Negative altitude WRT Earth, ze=-h, m",
    "Roll angle of body WRT Earth, phir, rad",
    "Mass of the aircraft, m, kg",
    "Target flight path angle gamma, tgammar, rad",
]
for var in var_list:
    print(f"{var} : min={all_data[var].min()}, max={all_data[var].max()}")

In [ ]:
t = np.arange(0, 3.1, 0.25)
v1 = 0.5*np.sin(1.5*t) + np.random.normal(0, 0.1, len(t))+0.5

font = {'size'   : 40}
matplotlib.rc('font', **font)

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.plot(t,v1, marker='o', markersize=10, color='black', linestyle='None')
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['bottom'].set_visible(False)
ax1.spines['left'].set_visible(False)
ax1.arrow(0, 0, 3, 0, head_width=0.2, color='black')
ax1.arrow(0, 0, 0, 1, head_width=0.2, color='black')
ax1.set_xlim(-0.1, 3.5)
ax1.set_ylim(-0.2, 1.4)
ax1.set_aspect('equal')
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('VAR')
ax1.set_yticklabels([])
ax1.set_xticklabels([])

plt.show()

fig.savefig(Path.cwd() / "figures" / "graphical_abstract_output_example.jpg")

print(v1)